In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
from simcore_sdk import node_ports
PORTS = node_ports.ports()

In [3]:
import pandas as pd
import matplotlib.pyplot as plt


def create_learned_model_input(path, plot_vs_tcnap):

    column_names = ['t_ms', 'CV', 'Vmax','M_mod', 'B_mod', 'tauSD']
    data = pd.read_csv(path, sep=',', names=column_names)



    dpi = 96
    height = 1024
    width = 1024
    fontsize = 16
    
    x_axis = data.CV
    if plot_vs_tcnap:
        x_axis = data.t_ms
        
        
    
    plt.figure(figsize=(width / dpi, height / dpi), dpi=dpi)    
    plt.subplot(4,1,1)
    plt.title(r'Learned Model Input Parameters', fontsize=20)
    plt.plot(x_axis, -1e12*data.Vmax, 'k.', markersize=5)
    plt.ylabel(r'$V_{MAX}$ $(\mu V)$', fontsize=fontsize)
    if plot_vs_tcnap:
        plt.xlim(0, max(data.t_ms))        
    else:
        plt.xlim(min(data.CV), 100)
        plt.xscale('log')
    plt.grid(which='both')

    plt.subplot(4,1,2)
    plt.plot(x_axis, data.M_mod, 'b.', markersize=5)
    plt.ylabel(r'M coeff $(\Delta_{Ist}/\Delta_\lambda$)', fontsize=fontsize)
    if plot_vs_tcnap:
        plt.xlim(0, max(data.t_ms))        
    else:
        plt.xlim(min(data.CV), 100)
        plt.xscale('log')
    plt.grid(which='both')

    plt.subplot(4,1,3)
    plt.plot(x_axis, data.B_mod, 'r.', markersize=5)
    plt.ylabel(r'B coeff $(mA)$', fontsize=fontsize)
    if plot_vs_tcnap:
        plt.xlim(0, max(data.t_ms))        
    else:
        plt.xlim(min(data.CV), 100)
        plt.xscale('log')
    plt.grid(which='both')

    plt.subplot(4,1,4)
    plt.plot(x_axis, data.tauSD, 'm.', markersize=5)
    plt.ylabel(r'$\tau_{SD}$ $(ms)$', fontsize=fontsize)
    plt.xlabel(r'Conduction Velocity $(m/s)$', fontsize=fontsize)
    if plot_vs_tcnap:
        plt.xlim(0, max(data.t_ms))        
    else:
        plt.xlim(min(data.CV), 100)
        plt.xscale('log')
    plt.grid(which='both')

    plt.show()


In [4]:
import ipywidgets as widgets

style = {"description_width": "initial"}

nerve_profile = widgets.Select(
    options=["Subject 1: Cervical Vagus", "Subject 2: Cervical Vagus", "Subject 2: Gastric Branch"],
    value="Subject 1: Cervical Vagus",
    # rows=10,
    description="Select:",
    disabled=False,
    style=style
)

load_input = widgets.Button(
    description="Load",
    disabled=False,
    button_style="success",
    align_self="right"
)

input_layout = widgets.VBox([nerve_profile, load_input])
input_label = widgets.Label("Select a Nerve Profile")
input_box = widgets.VBox([input_label, input_layout], layout=widgets.Layout(border="solid 1px"))

charge_phase_cb = widgets.Checkbox(
    value=False,
    description="Plot against Charge-Phase",
    disabled=False,
    style=style
)

time_cb = widgets.Checkbox(
    value=False,
    description="Plot CNAP versus Time (ms)",
    disabled=False,
    style=style
)

plot_options_layout = widgets.VBox([charge_phase_cb, time_cb])
plot_options_label = widgets.Label("Plot Options")
plot_options_box = widgets.VBox([plot_options_label, plot_options_layout], layout=widgets.Layout(border="solid 1px"))

horizontal_input_box = widgets.HBox([input_box, plot_options_box], layout=widgets.Layout(width="100%"))

out = widgets.Output(layout={'border': '1px solid black'})
vertical_box = widgets.VBox([horizontal_input_box, out])


#@out.capture()
def on_load_clicked(b):
        out.clear_output()
        with out:
            print("Load clicked.", nerve_profile.value)
            path = '/home/jovyan/input/input.csv'
            create_learned_model_input(path, plot_vs_tcnap=time_cb.value)
load_input.on_click(on_load_clicked)

display(vertical_box)

In [82]:
import pandas as pd
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt


def create_predicted_compound_nerve_action(cv_path, tst_path, vpred_path):
    data_cv = pd.read_csv(cv_path, sep=',', header=None)
    data_tst = pd.read_csv(tst_path, sep=',', header=None)
    data_vpred = pd.read_csv(vpred_path, sep=',', header=None)



    dpi = 96
    height = 1024
    width = 1024
    fontsize = 16
    
    #plt.title(r'Predicted Compound Nerve Action Potentials', fontsize=20)
    fig, [ax1, ax2] = plt.subplots(2, 1, figsize=(width / dpi, height / dpi), dpi=dpi, subplot_kw={'projection': '3d'})
    
    ax1.plot_wireframe(data_cv, data_tst, data_vpred, rstride=0, cstride=10)
    ax1.set_xlabel(r'CV (m/s)')
    ax1.set_ylabel(r'$I_{st}$ (mA)')
    ax1.set_zlabel(r'$V_{pred}$ $(\mu V)$')
    
    #
    ax1.set_xlim(left=min(data_cv), right=100)
    #ax1.set_xscale('log')
    #plt.grid(which='both')

    plt.subplot(2,1,2)
    plt.show()


In [5]:


start_ist = widgets.FloatText(
    value=0,
    description="Starting Ist (mA):",
    disabled=False,
    layout=widgets.Layout(width="200px"),
    style=style
)

end_ist = widgets.FloatText(
    value=1,
    description="Ending Ist (mA):",
    disabled=False,
    style=style
)

step_size_current = widgets.FloatText(
    value=0.01,
    description="Step Size (mA):",
    disabled=False,
    style=style
)

fixed_tst = widgets.FloatText(
    value=0.4,
    description="Fixed tst (ms):",
    disabled=False,
    style=style
)

predict_CNAPs_current = widgets.Button(
    description="Predict CNAPs",
    disabled=False,
    button_style="danger"
)

sweep_pulse_current_layout = widgets.VBox([start_ist, end_ist, step_size_current, fixed_tst, predict_CNAPs_current])
sweep_pulse_current_label = widgets.Label("Sweep Pulse Current")
sweep_pulse_current_box = widgets.VBox([sweep_pulse_current_label, sweep_pulse_current_layout], layout=widgets.Layout(border="solid 1px"))


start_tst = widgets.FloatText(
    value=0,
    description="Starting tst (mA):",
    disabled=False,
    style=style
)

end_tst = widgets.FloatText(
    value=1,
    description="Ending tst (mA):",
    disabled=False,
    style=style
)

step_size_duration = widgets.FloatText(
    value=0.01,
    description="Step Size (mA):",
    disabled=False,
    style=style
)

fixed_ist = widgets.FloatText(
    value=0.4,
    description="Fixed tst (ms):",
    disabled=False,
    style=style
)

predict_CNAPs_time = widgets.Button(
    description="Predict CNAPs",
    disabled=False,
    button_style="info",
    layout=widgets.Layout(align="right")
)

sweep_pulse_duration_layout = widgets.VBox([start_tst, end_tst, step_size_duration, fixed_ist, predict_CNAPs_time])
sweep_pulse_duration_label = widgets.Label("Sweep Pulse Duration")
sweep_pulse_duration_box = widgets.VBox([sweep_pulse_duration_label, sweep_pulse_duration_layout], layout=widgets.Layout(border="solid 1px"))


def on_current_clicked(b):
    print("Current clicked.", charge_phase_cb.value, time_cb.value, start_ist.value, end_ist.value, step_size_current.value, fixed_tst.value)
    create_predicted_compound_nerve_action(cv_path='/home/jovyan/input/CV_plot.csv',  tst_path='/home/jovyan/input/tst_plot.csv',   vpred_path='/home/jovyan/input/V_pred_plot.csv')
predict_CNAPs_current.on_click(on_current_clicked)


def on_time_clicked(b):
    print("Time clicked.", charge_phase_cb.value, time_cb.value, start_tst.value, end_tst.value, step_size_duration.value, fixed_ist.value)
predict_CNAPs_time.on_click(on_time_clicked)


parameters_layout = widgets.HBox([sweep_pulse_current_box, sweep_pulse_duration_box], layout=widgets.Layout(width="100%"))
parameters_layout